In [1]:
import tensorflow as tf
import os
import utils
import word2vec_utils

In [2]:
BATCH_SIZE = 128
VOCAB_SIZE = 50000
SKIP_WINDOW = 1 
VISUAL_FLD = 'visualization'

DOWNLOAD_URL = 'http://mattmahoney.net/dc/text8.zip'
EXPECTED_BYTES = 31344016
NUM_VISUALIZE = 3000        # number of tokens to visualize


In [17]:
def gen():
    yield from word2vec_utils.batch_gen(DOWNLOAD_URL, EXPECTED_BYTES, VOCAB_SIZE, 
                                        BATCH_SIZE, SKIP_WINDOW, VISUAL_FLD)

In [18]:
type([('UNK', -1)])

list

In [19]:
dataset = tf.data.Dataset.from_generator(gen, 
                                (tf.int32, tf.int32), 
                                (tf.TensorShape([BATCH_SIZE]), tf.TensorShape([BATCH_SIZE, 1])))

In [20]:
iterator = dataset.make_initializable_iterator()
center_words, target_words = iterator.get_next()

In [ ]:
with tf.Session() as sess:
    sess.run(iterator.initializer)   
    t1=sess.run([center_words, target_words])
    print(t1)

In [16]:
type(t1)

list

In [18]:
len(t1)

2

In [3]:
import zipfile
def read_data(file_path):
    """ Read data into a list of tokens 
    There should be 17,005,207 tokens
    """
    with zipfile.ZipFile(file_path) as f:
        words = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return words

In [4]:
local_dest = 'data/text8.zip'
utils.download_one_file(DOWNLOAD_URL, local_dest, EXPECTED_BYTES)
words = read_data(local_dest)
print(type(words))

data/text8.zip already exists
<class 'list'>


### There are 17,005,207 tokens if you count tokens by splitting the text by blank space.

In [11]:
words[0]

'anarchism'

In [12]:
words[1]

'originated'

In [13]:
len(words)

17005207

In [5]:
from collections import Counter
utils.safe_mkdir('visualization')
file = open(os.path.join('visualization', 'vocab.tsv'), 'w')
dictionary = dict()
count = [('UNK', -1)]
index = 0
count.extend(Counter(words).most_common(VOCAB_SIZE - 1))
print(type(count))
print(len(count))
print(count[0])
print(count[1])
print(count[2])

<class 'list'>
50000
('UNK', -1)
('the', 1061396)
('of', 593677)


In [6]:
for word, _ in count:
        dictionary[word] = index
       # print(word)
       # print(index)
        index += 1

In [7]:
len(dictionary)

50000

In [8]:
len(words)

17005207

In [9]:
def convert_words_to_index(words, dictionary):
    """ Replace each word in the dataset with its index in the dictionary """
    return [dictionary[word] if word in dictionary else 0 for word in words]

In [10]:
words[0]

'anarchism'

In [11]:
'anarchism' in dictionary

True

In [12]:
dictionary['anarchism']

5234

In [13]:
index_words = convert_words_to_index(words, dictionary)

In [14]:
index_words[0]

5234

In [50]:
import random
def generate_sample():
    for index,center in enumerate(index_words):
        context = random.randint(1, SKIP_WINDOW)
        print(index,center)
        print(context)
        for target in index_words[max(0, index - context): index]:
            yield center,target

In [57]:
single_gen = generate_sample()

In [58]:
next(single_gen)

0 5234
1
1 3081
1


(3081, 5234)

In [59]:
next(single_gen)

2 12
1


(12, 3081)

In [62]:
next(single_gen)

3 6
1


(6, 12)

In [44]:
def   yield5():
    yield 5
    yield 6

In [46]:
a

<generator object yield5 at 0x7fdce6fb2ba0>

In [47]:
next(a)

5

In [48]:
next(a)

6

In [49]:
next(a)

StopIteration: 

In [51]:
def   yield5():
    yield 5

In [52]:
a=yield5()

In [53]:
next(a)

5

In [54]:
next(a)

StopIteration: 

In [64]:
import random
def generate_sample():
    for index,center in enumerate(index_words):
        context = random.randint(1, SKIP_WINDOW)
        print(index,center)
        print(context)
        for target in index_words[max(0, index - context): index]:
            yield center,target
        for target in index_words[index + 1: index + context + 1]:
            yield center, target

In [65]:
single_gen = generate_sample()

In [69]:
index_words[:5]

[5234, 3081, 12, 6, 195]

In [66]:
next(single_gen)

0 5234
1


(5234, 3081)

In [67]:
next(single_gen)

1 3081
1


(3081, 5234)

In [68]:
next(single_gen)

(3081, 12)

In [70]:
next(single_gen)

2 12
1


(12, 3081)

In [71]:
next(single_gen)

(12, 6)